In [31]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import jdutil as jd
import datetime as dt
import sqlite3
import math
import matplotlib.pyplot as plt
import matplotlib.dates as dates

class aveHR:
    def __init__(self):
        self.tdist = 0
        self.hr = 0

    def step(self, hr, dist):
        self.hr += hr * dist
        self.tdist += dist

    def finalize(self):
        return self.hr / self.tdist

def jdate(day):
    datet = jd.jd_to_datetime(day)
    return datet.strftime( '%Y-%m-%d')

def pace(t, dist):
    secs = t / (dist * 0.001)
    mins = math.floor( secs / 60)
    secs = secs - mins * 60
    if round(secs) > 59:
        secs = 0
        mins = mins + 1    
    
    timet = dt.time( 0, mins, round(secs))
    return timet.strftime( '%M:%S')

class atleto:
    def __init__(self, fname):
        self.filename = fname
        
    def runs(self):
        conn = sqlite3.connect( self.filename)

        conn.create_function('jdate', 1, jdate)
        conn.create_function('pace', 2, pace)
        conn.create_aggregate('avehr', 2, aveHR)

        sqlite3.enable_callback_tracebacks(True) 

        runs = pd.read_sql_query("SELECT runs.id AS Id,\
                                    jdate(day) AS Date,\
                                    locations.name AS Location,\
                                    surfaces.name AS Surface\
                                    FROM runs\
                                    INNER JOIN locations ON locations.id=runs.location_id\
                                    INNER JOIN surfaces ON surfaces.id=runs.surface_id\
                                    ", conn)
        print(runs.head(10))
        print(runs.tail(10))
                
        splits = pd.read_sql_query("SELECT id AS Id,\
                                        run_id AS RunId,\
                                        dist AS Distance,\
                                        hr AS Heartrate,\
                                        t AS Time,\
                                        notes AS Notes\
                                        FROM run_splits", conn)
        print(splits.tail(15))      

file = '../website/atleto/data/atleto_data.atl'

a = atleto( file)
runs = a.runs()







     Id        Date           Location            Surface
0  5528  1983-09-18  Messdorf - Alfter  asphalt and trail
1  5529  1983-09-27  Messdorf - Alfter  asphalt and trail
2  5530  1983-10-16  Messdorf - Alfter  asphalt and trail
3  5531  1983-11-06  Messdorf - Alfter  asphalt and trail
4  5532  1983-11-13  Messdorf - Alfter  asphalt and trail
5  5549  1984-02-28  Messdorf - Alfter  asphalt and trail
6  5574  1984-11-06  Messdorf - Alfter  asphalt and trail
7  5576  1984-11-10   VL Bad Godesberg            unknown
8  5579  1984-11-17       VL Venusberg            unknown
9  5605  1985-01-02  Messdorf - Alfter  asphalt and trail
        Id        Date             Location         Surface
2569  8291  2019-01-29  Guildford / Johnson         asphalt
2570  8292  2019-02-01          Ioco bridge         asphalt
2571  8293  2019-01-31  Guildford / Johnson         asphalt
2572  8294  2019-02-02          Ioco bridge         asphalt
2573  8295  2019-02-03      Coquitlam River  mostly asphalt
25